## Cliente HTTP para consumir la API del modelo (Titanic)

En este notebook construimos un **cliente mínimo** con `requests` para enviar
peticiones a la API FastAPI creada en el notebook anterior (o a la versión
desplegada en Render).

**Objetivos**
- Probar el endpoint `POST /predict` con **tres payloads distintos**.
- (Opcional) variar el parámetro de consulta `confidence` para ver el efecto del umbral.
- Mostrar el **JSON enviado** y la **respuesta** de la API.

**Prerequisitos**
- Tener la API corriendo **local** o en **Render**.
  - Local: `http://127.0.0.1:8000`
  - Render (ejemplo): `https://producto-datos-lab-1.onrender.com`
- Verificar que responde `GET /healthz` y que `GET /docs` está disponible.

**Cómo usar este notebook**
1. Define `BASE_URL` al inicio (local o Render).
2. Ejecuta las celdas para construir la URL final (con o sin `confidence`).
3. Llama a `predict_passenger(payload)` con cada ejemplo y observa la salida.

> Tip: si recibes errores de conexión, revisa que el servidor esté activo y que
> `BASE_URL` apune al dominio correcto.


### Configuración básica
Definimos:
- `BASE_URL`: URL del servidor (local o Render).  
  - Local: `http://127.0.0.1:8000`  
  - Render: `https://producto-datos-lab-1.onrender.com`
- `ENDPOINT`: ruta del modelo (`/predict`).
- `CONFIDENCE` (opcional): umbral de decisión; si es `None` se usa el umbral guardado en el modelo.

La función `build_url(...)` arma la URL final (agregando `?confidence=` cuando aplica).

### Enviar un pasajero y obtener respuesta
`predict_passenger(passenger_dict, url, verbose)`:
- Envía un **JSON** con los campos crudos (`pclass, sex, age, sibsp, parch, fare, embarked`).
- Maneja errores de conexión y valida que el servidor devuelva **200**.
- Retorna el JSON de la API: `threshold_used`, `prob_survive`, `pred_class`, `pred_label`.

### Ejemplos individuales
Probamos dos pasajeros con características distintas para ver:
- La **probabilidad** de supervivencia.
- La **clase predicha** (`0/1`) y su **etiqueta** (`No sobrevive/Sobrevive`).

### Predicción por lote
`batch_predict([...])` recibe una lista de pasajeros y:
- Llama a la API por cada uno.
- Devuelve un **DataFrame** combinando entradas y salidas (útil para reportes rápidos).

**Notas rápidas**
- Si el request falla, revisa que la API responda en `GET /healthz` y que `BASE_URL` sea correcto.
- Para probar diferentes umbrales, cambia `CONFIDENCE` (p. ej., `0.5`, `0.6`) o pon `None`.
- Los valores deben cumplir las validaciones del servidor (Pydantic).


In [2]:
# === 02_client_titanic.ipynb ===
# Cliente para consumir la API del modelo de supervivencia del Titanic

import requests
import json
import pandas as pd

# 1) Configuración básica
BASE_URL = "http://127.0.0.1:8000"   # asegúrate de tener corriendo 01_server_titanic
ENDPOINT  = "/predict"
CONFIDENCE = 0.55                      # opcional: umbral; pon None para usar el del servidor

def build_url(base=BASE_URL, endpoint=ENDPOINT, confidence=CONFIDENCE):
    """Construye la URL con o sin el parámetro confidence."""
    url = base.rstrip("/") + endpoint
    if confidence is not None:
        # requests también permite pasar params=..., pero así ves la URL “final”
        url = f"{url}?confidence={confidence}"
    return url

FULL_URL = build_url()
print("URL:", FULL_URL)

# 2) Función genérica: enviar 1 pasajero y obtener respuesta
def predict_passenger(passenger_dict, url=FULL_URL, verbose=True):
    """
    passenger_dict: dict con las columnas crudas:
      pclass, sex, age, sibsp, parch, fare, embarked
    """
    try:
        resp = requests.post(url, json=passenger_dict, timeout=10)
    except requests.exceptions.RequestException as e:
        raise SystemExit(f"Error de conexión: {e}")

    if verbose:
        print("Status:", resp.status_code)

    if resp.status_code != 200:
        # Deja el detalle del servidor por si hay validaciones que no pasan
        raise ValueError(f"Error {resp.status_code}: {resp.text}")

    data = resp.json()
    # Espera: {"threshold_used":..., "prob_survive":..., "pred_class":..., "pred_label":...}
    if verbose:
        print(f"Umbral usado: {data['threshold_used']}")
        print(f"Prob. sobrevivir: {data['prob_survive']:.3f}")
        print(f"Predicción: {data['pred_class']} → {data['pred_label']}")
    return data

# 3) Ejemplos (ajusta según quieras)
helene = {
    "pclass": 1, "sex": "female", "age": 22, "sibsp": 0, "parch": 1,
    "fare": 80.0, "embarked": "C"
}
giles = {
    "pclass": 2, "sex": "male", "age": 21, "sibsp": 0, "parch": 0,
    "fare": 15.0, "embarked": "S"
}

print("\n--- Helene ---")
pred_h = predict_passenger(helene)

print("\n--- Giles ---")
pred_g = predict_passenger(giles)

# 4) Evaluación por lote (lista de pasajeros → DataFrame con resultados)
def batch_predict(passengers, url=FULL_URL):
    rows = []
    for p in passengers:
        data = predict_passenger(p, url=url, verbose=False)
        rows.append({
            **p,
            "threshold_used": data["threshold_used"],
            "prob_survive":  data["prob_survive"],
            "pred_class":    data["pred_class"],
            "pred_label":    data["pred_label"],
        })
    return pd.DataFrame(rows)

print("\n--- Batch ---")
df_out = batch_predict([helene, giles])
df_out

URL: http://127.0.0.1:8000/predict?confidence=0.55

--- Helene ---
Status: 200
Umbral usado: 0.55
Prob. sobrevivir: 0.960
Predicción: 1 → Sobrevive

--- Giles ---
Status: 200
Umbral usado: 0.55
Prob. sobrevivir: 0.272
Predicción: 0 → No sobrevive

--- Batch ---


,pclass,sex,age,sibsp,parch,fare,embarked,threshold_used,prob_survive,pred_class,pred_label
0,1,female,22,0,1,80.0,C,0.55,0.960318,1,Sobrevive
1,2,male,21,0,0,15.0,S,0.55,0.271564,0,No sobrevive
